In [ ]:
import numpy as np
import scipy.io.wavfile
from scipy.signal import convolve2d
import matplotlib.pyplot as plt
import sys
# %matplotlib inline
import librosa
import IPython.display as ipd


import os
import librosa.display
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shutil import copyfile
# os.chdir(r'.\Signal_processing\\')

In [ ]:
file = r'.\raster.mp3'
# file = r'.\A4_piano.wav'
# file = r'.\bass_synthetic_009-067-025.wav'
# file = r'.\data\audio_gammepno.wav'
# from pydub.utils import mediainfo
# info = mediainfo(file)
# # print(info)


signal, sr = librosa.load(file,16000,duration=2)
# times = np.linspace(0,1,sr)
# signal = np.sin(2*np.pi*2050*times) + np.sin(2*np.pi*1020*times+np.pi/3) + np.sin(2*np.pi*200*times+np.pi/3)
# signal = y

In [ ]:
plt.plot(signal)

In [ ]:
def get_stft(signal, sr, L, hop, Nfft):
    nb_trames = int((signal.size-L)/hop)
    freqs = np.linspace(0,sr/2,int(Nfft/2)+1)
    
    times = np.linspace(0,signal.size/sr,nb_trames)
    
    STFT = np.zeros((freqs.size,times.size),dtype='complex')
    
    for j in range(nb_trames):
        i0 = j*hop
        iend = min(i0+L,signal.size)
        
        signal_w = signal[i0:iend]
        fen = np.hanning(len(signal_w))
        signal_w = signal_w*fen
        STFT[:,j] = np.fft.rfft(signal_w,Nfft)#/(2*np.sum(fen))

    return STFT, freqs, times

In [ ]:
def rebuild_waveform(STFT, sr, L, hop, Nfft):
    N = hop*STFT.shape[1]+L
    fen = np.hanning(L)# définition de la fenetre d'analyse

    ws = fen; # définition de la fenêtre de synthèse
    y = np.zeros((N)) # signal de synthèse
    
    for u in np.arange(0,STFT.shape[1]).reshape(-1): # boucle sur les trames
        deb = u * hop  # début de trame
        fin = deb + L # fin de trame
        y[deb:fin]=y[deb:fin]+(np.real(np.fft.irfft(STFT[:,u])*ws))
    return(y)


In [ ]:
def stft_to_mel(STFT,nmel,Nfft,sr):

    hz_points = librosa.mel_frequencies(n_mels=nmel, fmin=0.0, fmax=sr/2,htk=True)

    fbank = librosa.filters.mel(sr=sr, n_fft=Nfft, n_mels=nmel,fmin=0,fmax=sr/2,htk=True,norm=False)
    
    STFT_mel_amp =np.dot(fbank, np.abs(STFT))
    STFT_mel_amp[STFT_mel_amp==0] = 1e-4 #On évite les 0 pour pouvoir prendre le log ensuite 
    
    STFT_mel_if = np.dot(fbank,instant_freq(np.angle(STFT)))

    return(STFT_mel_amp,STFT_mel_if,hz_points,fbank)




In [ ]:
def mel_to_stft(STFT_mel_amp,STFT_mel_if,fbank,griff=False,phase_init=False):
    
    fbank_inv = np.linalg.pinv(fbank)
    STFT_amp = fbank_inv @STFT_mel_amp
    STFT_if =  fbank_inv @STFT_mel_if
    STFT_phase = phase_from_if(STFT_if)
    
    
    # STFT_phase += np.reshape(np.angle(STFT)[:,0],(STFT.shape[0],1))@(np.ones((STFT.shape[1],1)).T)

    if griff==True:
        if phase_init ==False:
            STFT_phase,loss = griff_phase(STFT_amp, sr, L, hop, Nfft,None)
        else:
            STFT_phase,loss = griff_phase(STFT_amp, sr, L, hop, Nfft,STFT_phase)
        
    # STFT_phase += np.pi/3
    
    return(STFT_amp*np.exp(1j*STFT_phase),STFT_if)

In [ ]:
def instant_freq(phase):
    unwrapped_phase = np.unwrap(phase,axis=1)
    IF= np.pad(np.diff(unwrapped_phase,axis=1)/np.pi,([0,0],[1,0]))
    return(IF)

In [ ]:
def phase_from_if(IF):
    unwrapped_phase = np.cumsum(IF*np.pi,axis=1)
    return(unwrapped_phase)

In [ ]:
def get_closest_stft(X, sr,L,hop,Nfft):
    '''
    This function computes the signal x whose STFT is closest to the array X (the modified STFT), according to equation 6 in the article.
    This procedure is close, but not exactly equivalent, to the OLA method.
    INPUTS :
    - X : the modified STFT,
    - window: assumed window for the STFT,
    - Nfft : assumed number of FFT points for the STFT,
    - R : assumed hop size for the STFT,
    - plot : boolean, if True the STFT of the output signal is plotted,
    - Fe : sampling frequency
    OUTPUT : 
    - x : the estimated signal
    '''
    window = np.hanning(L)
    nb_frames = X.shape[0]
    Nw = len(window)
    signal_length = hop*STFT.shape[1]+L # TODO : FIX THIS, might be wrong
    x = np.empty(signal_length)
    for ind_frame,ind_time in enumerate(range(signal_length-Nw,hop)):
        x[ind_time:ind_time+Nw] += window*np.real(np.fft.rifft(X[ind_frame])) # TODO : maybe we need the negative frequencies as well here, be careful
    # TODO : the plot code
    return x

In [ ]:
def griff_phase(STFTM, sr, L, hop, Nfft,phase_init=None):
    
    N = hop*STFTM.shape[1]+L

    nb_frames = STFTM.shape[0]
    signal_length = hop*STFTM.shape[1]+L  # TODO : FIX THIS, might be wrong

    N_iterations = 50# number of iterations 
    loss = np.empty(N_iterations) # to show the evolution of the loss (MSE) over the iterations

    #TODO : fix the initial signal's length
    # x_recovered = np.random.rand(N) # Initial guess for the reconstructed signal, a Gaussian noise in the original article
    if phase_init is None:
        STFT_hat =STFTM*np.exp( 1j*np.random.rand(STFTM.shape[0],STFTM.shape[1])*np.pi/2)
    else:
        STFT_hat =STFTM*np.exp( 1j*phase_init)
    
    for ind_iteration in range(N_iterations):
        x_recovered = rebuild_waveform(STFT_hat, sr, L, hop, Nfft)
        
        STFT_temp,freq,times = get_stft(x_recovered, sr, L, hop, Nfft)
        STFT_hat = STFTM *np.exp(1j* np.angle(STFT_temp))
        
        loss[ind_iteration] = np.linalg.norm(np.abs(STFT_temp)-STFTM)
    return(np.angle(STFT_hat),loss)

In [ ]:
def plot_spectrogram(spectre, col, row, labelCol='', labelRow='',vmin=None,vmax=None,ax=None,fig=None):
    X,Y = np.meshgrid(row,col)
    if ax==None:
        fig,ax=plt.subplots()
    im = ax.pcolor(X,Y,spectre,cmap='magma',vmin=vmin,vmax=vmax)
    add_colorbar(fig,ax,im)
    ax.set_xlabel(labelRow)
    ax.set_ylabel(labelCol)
    return

def add_colorbar(fig,ax,im):
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im, cax=cax, orientation='vertical')

In [ ]:
Nfft =1024
Nfft2 = int(Nfft/2)+1
hop =int(Nfft/4)
L = Nfft
nmel= int(Nfft2)
 
STFT,freqs,times = get_stft(signal, sr, L, hop, Nfft)



STFT_mel_amp,STFT_mel_if,hz_points,fbank = stft_to_mel(STFT,nmel,Nfft,sr)


# STFT_lib = librosa.stft(signal, Nfft, hop_length=hop, win_length=L, window='hann')
# STFT_lib_mel = librosa.feature.melspectrogram(S=np.abs(STFT_lib), sr=sr, hop_length=hop,n_fft=Nfft,n_mels=nmel,win_length=L,window='hann')

# STFT_rec,STFT_if = mel_to_stft(STFT_mel_amp,STFT_mel_if,fbank)


#BackWard
STFT_rec2,STFT_if = mel_to_stft(STFT_mel_amp,STFT_mel_if,fbank,griff=True)
rec = rebuild_waveform(STFT_rec, sr, L, hop, Nfft)

# STFT_rec3,STFT_if = mel_to_stft(STFT_mel_amp,STFT_mel_if,fbank,griff=True,phase_init=True)


rec2 = rebuild_waveform(STFT_rec2, sr, L, hop, Nfft)
rec3 = rebuild_waveform(STFT_rec3, sr, L, hop, Nfft)




f,ax = plt.subplots(1,2,figsize=(20,5))
for a in ax.flatten():
    a.set_ylim(0,5000)

# ax[0,0].set_title('Linear - Librosa')
# im0=librosa.display.specshow(20*np.log10(np.abs(STFT_lib)/np.max(np.abs(STFT_lib))), sr=sr, x_axis='time', y_axis='linear',ax=ax[0,0])
# ax[0,0].set_ylim(0,6000)
# add_colorbar(f,ax[0,0],im0)
# # plt.colorbar(format='%+2.0f dB');



ax[0].set_title('Linear - Home made')
plot_spectrogram(20*np.log10(np.abs(STFT)/np.max(np.abs(STFT))), freqs, times, 'Frequency [Hz]', 'Time [sec]',ax=ax[0],fig=f)




# ax[1,0].set_title('Mel Scale - Librosa')
# im1=librosa.display.specshow(20*np.log10(np.abs(STFT_lib_mel)/np.max(np.abs(STFT_lib_mel))), sr=sr, x_axis='time', y_axis='mel',ax=ax[1,0]);
# add_colorbar(f,ax[1,0],im1)



ax[1].set_title('Mel Scale - Home made')
plot_spectrogram(20*np.log10(np.abs(STFT_mel_amp)/np.max(np.abs(STFT_mel_amp))), hz_points, times, 'Frequency [Hz]', 'Time [sec]',ax=ax[1],fig=f)




f,ax = plt.subplots(2,2,figsize=(20,10))
for a in ax.flatten():
    a.set_ylim(0,5000)


ax[0,0].set_title('Phase')
plot_spectrogram(np.angle(STFT), freqs, times, 'Frequency [Hz]', 'Time [sec]',ax=ax[0,0],fig=f)


ax[0,1].set_title('Unwrapped Phase')
plot_spectrogram(np.unwrap(np.angle(STFT),axis=1), freqs, times, 'Frequency [Hz]', 'Time [sec]',ax=ax[0,1],fig=f)

ax[1,0].set_title('Instant Frequency')
inst_freq = instant_freq(np.angle(STFT))
plot_spectrogram(instant_freq(np.angle(STFT)), freqs, times, 'Frequency [Hz]', 'Time [sec]',ax=ax[1,0],fig=f)


ax[1,1].set_title('Instant Frequency - Mel')
plot_spectrogram(STFT_mel_if, hz_points, times, 'Frequency [Hz]', 'Time [sec]',ax=ax[1,1],fig=f)



f,ax = plt.subplots(4,2,figsize=(20,20))
for a in ax.flatten():
    a.set_ylim(0,5000)

ax[0,0].set_title('Linear - Initial')
plot_spectrogram(20*np.log10(np.abs(STFT)), freqs, times, 'Frequency [Hz]', 'Time [sec]',ax=ax[0,0],fig=f)

ax[0,1].set_title('Linear - Reconstructed')
plot_spectrogram(20*np.log10(np.abs(STFT_rec)), freqs, times, 'Frequency [Hz]', 'Time [sec]',ax=ax[0,1],fig=f)


ax[1,0].set_title('Linear IF')
plot_spectrogram(instant_freq(np.angle(STFT)), freqs, times, 'Frequency [Hz]', 'Time [sec]',ax=ax[1,0],fig=f)

ax[1,1].set_title('IF - Reconstructed')
plot_spectrogram(STFT_if, freqs, times, 'Frequency [Hz]', 'Time [sec]',ax=ax[1,1],fig=f)


ax[2,0].set_title('Linear Phase')
plot_spectrogram(phase_from_if(instant_freq(np.angle(STFT))), freqs, times, 'Frequency [Hz]', 'Time [sec]',ax=ax[2,0],fig=f)

ax[2,1].set_title('Rebuild Phase')
plot_spectrogram(phase_from_if(STFT_if), freqs, times, 'Frequency [Hz]', 'Time [sec]',ax=ax[2,1],fig=f)





ax[3,0].set_title('Linear Phase')
plot_spectrogram(np.angle(STFT), freqs, times, 'Frequency [Hz]', 'Time [sec]',ax=ax[3,0],fig=f)

ax[3,1].set_title('Rebuild Phase')
plot_spectrogram(np.angle(STFT_rec), freqs, times, 'Frequency [Hz]', 'Time [sec]',ax=ax[3,1],fig=f)


f,ax = plt.subplots(4,1,figsize=(20,20))
ax[0].plot(signal[:20000]/np.max(np.abs(signal)))
ax[0].set_title('Signal Original')
ax[1].plot(rec[:20000]/np.max(np.abs(rec)))
ax[1].set_title('Signal Reconstruit avec la fréquence instantannée')
ax[2].plot(rec2[:20000]/np.max(np.abs(rec2)))
ax[2].set_title('Signal Reconstruit avec griff')
ax[3].plot(rec3[:20000]/np.max(np.abs(rec3)))
ax[3].set_title('Signal Reconstruit avec griff mais initialisation IF')


In [ ]:
f,ax = plt.subplots(4,1,figsize=(20,20))
ax[0].plot(signal/np.max(np.abs(signal)))
ax[0].set_title('Signal Original')
ax[1].plot(rec/np.max(np.abs(rec)))
ax[1].set_title('Signal Reconstruit avec la fréquence instantannée')
ax[2].plot(rec2/np.max(np.abs(rec2)))
ax[2].set_title('Signal Reconstruit avec griff')
ax[3].plot(rec3/np.max(np.abs(rec3)))
ax[3].set_title('Signal Reconstruit avec griff mais initialisation IF')

plt.figure()
plt.plot(loss)
plt.title('Loss')

In [ ]:
plt.plot(rec2[0:]/np.max(rec2))
plt.plot(rec3[0:]/np.max(rec3))

plt.plot(signal/np.max(signal),alpha=0.3)
plt.xlim(5000,5500)

In [ ]:
ipd.Audio(signal, rate=sr)



In [ ]:
ipd.Audio(rec/np.max(rec), rate=sr)

In [ ]:
ipd.Audio(rec2/np.max(rec2), rate=sr)

In [ ]:
ipd.Audio(rec3/np.max(rec3), rate=sr)